In [1]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/Neural_CDE/applications-of-NDE

%load_ext autoreload
%autoreload 2

import sys
sys.path.append(f'/content/drive/My Drive/Neural_CDE/applications-of-NDE')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Neural_CDE/applications-of-NDE


In [9]:
!pip install gymnasium

!pip install wandb

#Setting up pygame for colab
!python -m pip install pygame
import os
os.environ['SDL_VIDEODRIVER']='dummy'
import pygame
pygame.display.set_mode((640,480))


# Graphics and plotting.
print('Installing mediapy:')
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)
!pip install -q mediapy
import mediapy as media
import matplotlib.pyplot as plt


#for neural CDE
!pip install torchcde torchdiffeq

import torchcde
import torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 20.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 5.8 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=55e0a7f31e67141c96362343c51d6a7a0a39dec5bcba072c4ab878e3c11b18d2
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
pygame 2.3.0 (SDL 2.24.2, Python 3.9.16)
Hello fro

In [10]:
!wandb login

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
!python main.py

pygame 2.3.0 (SDL 2.24.2, Python 3.9.16)
Hello from the pygame community. https://www.pygame.org/contribute.html
Degenerated State space dim:  152
Number of parameters in baseline:  2808
Number of parameters in test model:  23552
Generating data...
Generating data for policy 1...
Generating data for policy 2...
Generating data for policy 3...
Generating data for policy 4...
Generating data for policy 5...



wandb: Currently logged in as: josssdan. Use `wandb login --relogin` to force relogin
wandb: Tracking run with wandb version 0.14.0
wandb: Run data is saved locally in /content/drive/MyDrive/Neural_CDE/applications-of-NDE/wandb/run-20230326_185812-6gds5i0r
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run hardy-capybara-5
wandb: ⭐️ View project at https://wandb.ai/josssdan/NDE_as_Mental_Models
wandb: 🚀 View run at https://wandb.ai/josssdan/NDE_as_Mental_Models/runs/6gds5i0r
Baseline model
Training model...
Epoch: 0 Training loss: 0.22228285670280457
Epoch: 1 Traini

In [ ]:
import torch.nn as nn
import torchdiffeq

In [ ]:
class DegeneratedMarkovStateEvolver(torch.nn.Module):
    def __init__(self, state_dim,hidden_channels):
        super(DegeneratedMarkovStateEvolver, self).__init__()

        self.net = nn.Sequential(
            nn.Linear(state_dim, hidden_channels),
            nn.Tanh(),
            nn.Linear(hidden_channels, state_dim),
        )

        for m in self.net.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight, mean=0, std=0.1)
                nn.init.constant_(m.bias, val=0)

    def forward(self, t, y):
        return self.net(y)

In [ ]:
test = DegeneratedMarkovStateEvolver(10,10)

In [ ]:
torchdiffeq.odeint(test, torch.tensor(5.0).repeat(100,10), torch.stack((torch.tensor(0.0),torch.tensor(5.0)))).shape

torch.Size([2, 100, 10])

In [ ]:
torch.stack((torch.tensor(0.0),torch.tensor(5.0)))

tensor([0., 5.])